In [35]:
import pandas as pd
train=pd.read_csv("out/train.csv")
test=pd.read_csv("out/test.csv")
elo_df=pd.read_csv("out/elo.csv")

In [2]:
train.head()

,home_team,away_team,elo_diff,h2h_index,result
0,Arsenal,West Ham,243,1.68,H
1,West Ham,Fulham,-32,2.08,D
2,Fulham,Leeds,60,2.73,D
3,Brentford,Sunderland,264,6.74,H
4,Brentford,Tottenham,37,1.12,D


In [3]:
test.head()

,Home,Away,home_team_elo,away_team_elo,home_stadium_win_pct,away_stadium_win_pct,h2h_bias
0,7,1,1993,1808,64,26,12
1,495,9,1873,1869,42,26,-3
2,2,502,1827,1782,35,18,-9
3,500,14,1547,1750,32,24,0
4,15,8,1774,1730,54,20,12


In [36]:
elo_df.head()

,Team,Team ID,Elo Rating,awayw_pct,homew_pct
0,Man City,12,1960,42,60
1,Sunderland,500,1547,18,32
2,Tottenham,15,1774,33,54
3,Liverpool,7,1993,43,64
4,Nottm Forest,487,1803,26,37


In [5]:
X=train[['elo_diff','h2h_index']]
y=train['result']

In [78]:
y.value_counts()

result
H    1336
A     873
D     791
Name: count, dtype: int64

In [108]:
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, log_loss

y_num=y.map({'H': 0, 'D': 1, 'A': 2})
X_train, X_test, y_train, y_test=train_test_split(X,y_num,test_size=0.2,random_state=42)
cat_clf=CatBoostClassifier(iterations=100, learning_rate=0.05, depth=16, loss_function='MultiClass',
                           eval_metric='Accuracy',class_weights={0:0.8,1:1.2,2:1},verbose=0, random_seed=42)

cat_clf.fit(X_train, y_train)
y_pred=cat_clf.predict(X_test)
print(accuracy_score(y_test, y_pred))
y_proba=cat_clf.predict_proba(X_test)
print(log_loss(y_test, y_proba))


0.43666666666666665
1.0697094570331067


In [109]:
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, log_loss

y_num=y.map({'H': 0, 'D': 1, 'A': 2})
X_train, X_test, y_train, y_test=train_test_split(X,y_num,test_size=0.2,random_state=42)
cat_clfp=CatBoostClassifier(iterations= 964, depth= 5, 
learning_rate= 0.2499323695909535, l2_leaf_reg= 1.4864392090173522, 
bagging_temperature= 1.2660516317554948, border_count= 206, 
random_strength= 3.3236428677766416)

cat_clfp.fit(X_train, y_train)
y_pred=cat_clfp.predict(X_test)
print(accuracy_score(y_test, y_pred))
y_proba=cat_clfp.predict_proba(X_test)
print(log_loss(y_test, y_proba))


0:	learn: 1.0676544	total: 1.06ms	remaining: 1.02s
1:	learn: 1.0473985	total: 2ms	remaining: 964ms
2:	learn: 1.0343124	total: 2.6ms	remaining: 834ms
3:	learn: 1.0249963	total: 3.19ms	remaining: 766ms
4:	learn: 1.0189921	total: 3.83ms	remaining: 735ms
5:	learn: 1.0153692	total: 4.67ms	remaining: 745ms
6:	learn: 1.0125091	total: 5.34ms	remaining: 730ms
7:	learn: 1.0097273	total: 5.9ms	remaining: 705ms
8:	learn: 1.0077868	total: 6.51ms	remaining: 691ms
9:	learn: 1.0070376	total: 6.85ms	remaining: 653ms
10:	learn: 1.0046458	total: 7.42ms	remaining: 643ms
11:	learn: 1.0036102	total: 8.14ms	remaining: 646ms
12:	learn: 1.0027177	total: 8.81ms	remaining: 645ms
13:	learn: 1.0016765	total: 9.52ms	remaining: 646ms
14:	learn: 1.0008318	total: 10.1ms	remaining: 638ms
15:	learn: 0.9995411	total: 10.6ms	remaining: 629ms
16:	learn: 0.9977882	total: 11.1ms	remaining: 621ms
17:	learn: 0.9963772	total: 11.7ms	remaining: 615ms
18:	learn: 0.9957645	total: 12.2ms	remaining: 609ms
19:	learn: 0.9949902	total:

In [110]:
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, log_loss

y_num=y.map({'H': 0, 'D': 1, 'A': 2})
X_train, X_test, y_train, y_test=train_test_split(X,y_num,test_size=0.2,random_state=42)
cat_clf_combo=CatBoostClassifier(iterations= 964, depth= 5, 
learning_rate= 0.2499323695909535, l2_leaf_reg= 1.4864392090173522, 
bagging_temperature= 1.2660516317554948, border_count= 206, 
random_strength= 3.3236428677766416, loss_function='MultiClass',
                           eval_metric='Accuracy',class_weights={0:0.8,1:1.2,2:1},verbose=0, random_seed=42)

cat_clf_combo.fit(X_train, y_train)
y_pred=cat_clf_combo.predict(X_test)
print(accuracy_score(y_test, y_pred))
y_proba=cat_clf_combo.predict_proba(X_test)
print(log_loss(y_test, y_proba))


0.415
1.227231050609917


In [111]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(
    n_estimators=100,
    max_depth=32,             # limits complexity
    min_samples_leaf=10,       # avoids overfitting
    max_features="sqrt",      # increases randomness
    class_weight={0:0.9,1:1.1,2:1},  # handles skew
    random_state=42
)
rf.fit(X_train, y_train)
y_pred=rf.predict(X_test)
print(accuracy_score(y_test, y_pred))
y_proba=rf.predict_proba(X_test)
print(log_loss(y_test, y_proba))

0.4483333333333333
1.0629555683681655


In [96]:
%pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 4.4 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5/5 [optuna]2m4/5 [optuna]
Note: you may need to restart the kernel to use updated packages.


In [97]:
import optuna
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss, accuracy_score
import numpy as np

# ---------------------------
# Train/Test Split
# ---------------------------
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

train_pool = Pool(X_train, y_train)
val_pool = Pool(X_val, y_val)

# ---------------------------
# Objective Function
# ---------------------------
def objective(trial):
    params = {
        "iterations": trial.suggest_int("iterations", 300, 1500),
        "depth": trial.suggest_int("depth", 4, 10),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1, 10, log=True),
        "bagging_temperature": trial.suggest_float("bagging_temperature", 0, 5),
        "border_count": trial.suggest_int("border_count", 32, 255),
        "random_strength": trial.suggest_float("random_strength", 0.5, 5),
        "loss_function": "MultiClass",
        "eval_metric": "MultiClass",
        "random_seed": 42,
        "verbose": 0
    }

    model = CatBoostClassifier(**params)
    model.fit(train_pool, eval_set=val_pool, early_stopping_rounds=50, verbose=0)

    preds = model.predict_proba(X_val)
    loss = log_loss(y_val, preds)

    return loss

# ---------------------------
# Run Optimization
# ---------------------------
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=30)  # increase trials for better results

print("Best Trial:", study.best_trial.params)

# ---------------------------
# Train Final Model with Best Params
# ---------------------------
best_params = study.best_trial.params
final_model = CatBoostClassifier(**best_params)
final_model.fit(Pool(X, y), verbose=100)



/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-08-19 18:26:10,845] A new study created in memory with name: no-name-9262a544-94eb-4b41-b40d-faa0c6b507e1
[I 2025-08-19 18:26:10,984] Trial 0 finished with value: 1.012013755124681 and parameters: {'iterations': 549, 'depth': 8, 'learning_rate': 0.16220097030216835, 'l2_leaf_reg': 1.163588804764033, 'bagging_temperature': 3.0476875860758934, 'border_count': 219, 'random_strength': 4.572585374973545}. Best is trial 0 with value: 1.012013755124681.
[I 2025-08-19 18:26:11,096] Trial 1 finished with value: 1.0097401316413117 and parameters: {'iterations': 769, 'depth': 7, 'learning_rate': 0.04217112057993669, 'l2_leaf_reg': 9.482057453388961, 'bagging_temperature': 3.963049215143299, 'border_count': 92

Best Trial: {'iterations': 964, 'depth': 5, 'learning_rate': 0.2499323695909535, 'l2_leaf_reg': 1.4864392090173522, 'bagging_temperature': 1.2660516317554948, 'border_count': 206, 'random_strength': 3.3236428677766416}
0:	learn: 1.0665838	total: 575us	remaining: 554ms
100:	learn: 0.9418232	total: 58.6ms	remaining: 501ms
200:	learn: 0.8897501	total: 114ms	remaining: 433ms
300:	learn: 0.8495461	total: 170ms	remaining: 374ms
400:	learn: 0.8162798	total: 229ms	remaining: 322ms
500:	learn: 0.7878273	total: 285ms	remaining: 264ms
600:	learn: 0.7629658	total: 343ms	remaining: 207ms
700:	learn: 0.7395608	total: 400ms	remaining: 150ms
800:	learn: 0.7186147	total: 460ms	remaining: 93.5ms
900:	learn: 0.6994435	total: 511ms	remaining: 35.7ms
963:	learn: 0.6873219	total: 546ms	remaining: 0us


In [40]:
import numpy as np

K = 400.0 / np.log(10.0)  # ≈ 173.7178

def logit(p, eps=1e-6):
    p = np.clip(p, eps, 1 - eps)
    return np.log(p / (1 - p))

def eb_shrink(p, n, p_bar, lam=25):
    return (n*p + lam*p_bar) / (n + lam)

def compute_adjusted_elo_diff(
    df,
    HFA=50.0,
    p_bar_home=0.475,
    p_bar_away=0.29,
    beta_h=0.5,
    beta_a=0.5,
    lam=25,
    clip_boost=60.0
):
    n_home = df.get('n_home_games', 38)
    n_away = df.get('n_away_games', 38)

    p_home_eb = eb_shrink(df['home_stadium_win_pct'].values, np.array(n_home), p_bar_home, lam)
    p_away_eb = eb_shrink(df['away_stadium_win_pct'].values, np.array(n_away), p_bar_away, lam)

    d_home = K * beta_h * (logit(p_home_eb) - logit(p_bar_home))
    d_away = K * beta_a * (logit(p_away_eb) - logit(p_bar_away))

    d_home = np.clip(d_home, -clip_boost, clip_boost)
    d_away = np.clip(d_away, -clip_boost, clip_boost)

    return (df['home_team_elo'].values + HFA + d_home) - (df['away_team_elo'].values + d_away)


test['elo_dif']  = compute_adjusted_elo_diff(test)


In [41]:
test.head()

,Home,Away,home_team_elo,away_team_elo,home_stadium_win_pct,away_stadium_win_pct,h2h_bias,elo_dif
0,7,1,1993,1808,64,26,12,235.0
1,495,9,1873,1869,42,26,-3,54.0
2,2,502,1827,1782,35,18,-9,95.0
3,500,14,1547,1750,32,24,0,-153.0
4,15,8,1774,1730,54,20,12,94.0


In [44]:
test=test.rename(columns={
    'elo_dif':'elo_diff',
    'h2h_bias' : 'h2h_index'
})

In [112]:
import numpy as np

# Elo update function
def update_elo(elo_home, elo_away, result, k=20):
    """
    Updates Elo ratings after a match.
    result: 'H', 'A', or 'D' (home, away, draw)
    k: K-factor (controls update magnitude)
    """
    # Expected probabilities (logistic model)
    exp_home = 1 / (1 + 10 ** ((elo_away - elo_home) / 400))
    exp_away = 1 - exp_home

    # Actual outcomes
    if result == 0:
        score_home, score_away = 1, 0
    elif result == 2:
        score_home, score_away = 0, 1
    else:  # Draw
        score_home, score_away = 0.5, 0.5

    # Update ratings
    new_home = elo_home + k * (score_home - exp_home)
    new_away = elo_away + k * (score_away - exp_away)

    return new_home, new_away


# ID → Team mapping
id_map = dict(zip(elo_df['Team ID'], elo_df['Team']))

# Initialize points
points = {
    'Liverpool': 0,
    'Man City': 0,
    'Arsenal ': 0,
    'Man Utd': 0,
    'Chelsea': 0,
    'Tottenham ': 0,
    'Aston Villa': 0,
    'Brighton': 0,
    'Newcastle Utd': 0,
    'Brentford': 0,
    'Fulham': 0,
    'Crystal Palace': 0,
    'Wolves': 0,
    'West Ham': 0,
    'Nottm Forest': 0,
    'Bournemouth ': 0,
    'Everton': 0,
    'Sunderland': 0,
    'Burnley': 0,
    'Leeds': 0
}

# Create a dict to hold current Elo ratings (from df)
elos = {}
for _, row in elo_df.iterrows():
    elos[row['Team']] = row['Elo Rating']

# Extract features and predict
X_new = test[['elo_diff','h2h_index']]
y_new = cat_clf.predict(X_new).flatten()
test['result'] = y_new

# Counters
h_ct, a_ct, d_ct = 0, 0, 0

# Simulate season
for index, row in test.iterrows():
    home_team = id_map.get(row['Home'], row['Home'])
    away_team = id_map.get(row['Away'], row['Away'])
    result = row['result']

    # Use current Elo values
    elo_home = float(elos[home_team])
    elo_away = float(elos[away_team])

    print(f"Match: {home_team} ({elo_home:.1f}) vs {away_team} ({elo_away:.1f}), Result: {'H' if result == 0 else 'A' if result == 2 else 'D'}")

    # Update points
    if result == 0:
        points[home_team] += 3
        h_ct += 1
    elif result == 2:
        points[away_team] += 3
        a_ct += 1
    elif result == 1:
        points[home_team] += 1
        points[away_team] += 1
        d_ct += 1

    # Update Elo ratings
    new_home, new_away = update_elo(elo_home, elo_away, result, k=20)
    elos[home_team] = new_home
    elos[away_team] = new_away

# Final table
sorted_points = dict(sorted(points.items(), key=lambda item: item[1], reverse=True))
print("\n--- Final Points Table ---")
i=1
for team, pts in sorted_points.items():
    print(f"{i}. {team}: {pts} points (Elo {elos[team]:.1f})")
    i+=1

print(f"\nHome Wins: {h_ct}, Away Wins: {a_ct}, Draws: {d_ct}")


Match: Liverpool (1993.0) vs Bournemouth  (1808.0), Result: H
Match: Aston Villa (1873.0) vs Newcastle Utd (1869.0), Result: D
Match: Brighton (1827.0) vs Fulham (1782.0), Result: H
Match: Sunderland (1547.0) vs West Ham (1750.0), Result: A
Match: Tottenham  (1774.0) vs Burnley (1730.0), Result: H
Match: Wolves (1736.0) vs Man City (1960.0), Result: A
Match: Nottm Forest (1803.0) vs Brentford (1811.0), Result: D
Match: Chelsea (1903.0) vs Crystal Palace (1835.0), Result: H
Match: Man Utd (1799.0) vs Arsenal  (1993.0), Result: A
Match: Leeds (1722.0) vs Everton (1794.0), Result: D
Match: West Ham (1754.7) vs Chelsea (1911.1), Result: A
Match: Man City (1964.3) vs Tottenham  (1782.7), Result: H
Match: Bournemouth  (1802.9) vs Wolves (1731.7), Result: H
Match: Brentford (1810.8) vs Aston Villa (1872.9), Result: A
Match: Burnley (1721.3) vs Sunderland (1542.3), Result: H
Match: Arsenal  (1997.9) vs Leeds (1724.0), Result: H
Match: Crystal Palace (1826.9) vs Nottm Forest (1803.2), Result: D

In [113]:
import numpy as np

# Elo update function
def update_elo(elo_home, elo_away, result, k=20):
    """
    Updates Elo ratings after a match.
    result: 'H', 'A', or 'D' (home, away, draw)
    k: K-factor (controls update magnitude)
    """
    # Expected probabilities (logistic model)
    exp_home = 1 / (1 + 10 ** ((elo_away - elo_home) / 400))
    exp_away = 1 - exp_home

    # Actual outcomes
    if result == 0:
        score_home, score_away = 1, 0
    elif result == 2:
        score_home, score_away = 0, 1
    else:  # Draw
        score_home, score_away = 0.5, 0.5

    # Update ratings
    new_home = elo_home + k * (score_home - exp_home)
    new_away = elo_away + k * (score_away - exp_away)

    return new_home, new_away


# ID → Team mapping
id_map = dict(zip(elo_df['Team ID'], elo_df['Team']))

# Initialize points
points = {
    'Liverpool': 0,
    'Man City': 0,
    'Arsenal ': 0,
    'Man Utd': 0,
    'Chelsea': 0,
    'Tottenham ': 0,
    'Aston Villa': 0,
    'Brighton': 0,
    'Newcastle Utd': 0,
    'Brentford': 0,
    'Fulham': 0,
    'Crystal Palace': 0,
    'Wolves': 0,
    'West Ham': 0,
    'Nottm Forest': 0,
    'Bournemouth ': 0,
    'Everton': 0,
    'Sunderland': 0,
    'Burnley': 0,
    'Leeds': 0
}

# Create a dict to hold current Elo ratings (from df)
elos = {}
for _, row in elo_df.iterrows():
    elos[row['Team']] = row['Elo Rating']

# Extract features and predict
X_new = test[['elo_diff','h2h_index']]
y_new = cat_clfp.predict(X_new).flatten()
test['result'] = y_new

# Counters
h_ct, a_ct, d_ct = 0, 0, 0

# Simulate season
for index, row in test.iterrows():
    home_team = id_map.get(row['Home'], row['Home'])
    away_team = id_map.get(row['Away'], row['Away'])
    result = row['result']

    # Use current Elo values
    elo_home = float(elos[home_team])
    elo_away = float(elos[away_team])

    print(f"Match: {home_team} ({elo_home:.1f}) vs {away_team} ({elo_away:.1f}), Result: {'H' if result == 0 else 'A' if result == 2 else 'D'}")

    # Update points
    if result == 0:
        points[home_team] += 3
        h_ct += 1
    elif result == 2:
        points[away_team] += 3
        a_ct += 1
    elif result == 1:
        points[home_team] += 1
        points[away_team] += 1
        d_ct += 1

    # Update Elo ratings
    new_home, new_away = update_elo(elo_home, elo_away, result, k=20)
    elos[home_team] = new_home
    elos[away_team] = new_away

# Final table
sorted_points = dict(sorted(points.items(), key=lambda item: item[1], reverse=True))
print("\n--- Final Points Table ---")
i=1
for team, pts in sorted_points.items():
    print(f"{i}. {team}: {pts} points (Elo {elos[team]:.1f})")
    i+=1

print(f"\nHome Wins: {h_ct}, Away Wins: {a_ct}, Draws: {d_ct}")


Match: Liverpool (1993.0) vs Bournemouth  (1808.0), Result: H
Match: Aston Villa (1873.0) vs Newcastle Utd (1869.0), Result: D
Match: Brighton (1827.0) vs Fulham (1782.0), Result: H
Match: Sunderland (1547.0) vs West Ham (1750.0), Result: A
Match: Tottenham  (1774.0) vs Burnley (1730.0), Result: H
Match: Wolves (1736.0) vs Man City (1960.0), Result: H
Match: Nottm Forest (1803.0) vs Brentford (1811.0), Result: D
Match: Chelsea (1903.0) vs Crystal Palace (1835.0), Result: H
Match: Man Utd (1799.0) vs Arsenal  (1993.0), Result: A
Match: Leeds (1722.0) vs Everton (1794.0), Result: D
Match: West Ham (1754.7) vs Chelsea (1911.1), Result: A
Match: Man City (1944.3) vs Tottenham  (1782.7), Result: H
Match: Bournemouth  (1802.9) vs Wolves (1751.7), Result: H
Match: Brentford (1810.8) vs Aston Villa (1872.9), Result: D
Match: Burnley (1721.3) vs Sunderland (1542.3), Result: H
Match: Arsenal  (1997.9) vs Leeds (1724.0), Result: H
Match: Crystal Palace (1826.9) vs Nottm Forest (1803.2), Result: A

In [115]:
import numpy as np

# Elo update function
def update_elo(elo_home, elo_away, result, k=20):
    """
    Updates Elo ratings after a match.
    result: 'H', 'A', or 'D' (home, away, draw)
    k: K-factor (controls update magnitude)
    """
    # Expected probabilities (logistic model)
    exp_home = 1 / (1 + 10 ** ((elo_away - elo_home) / 400))
    exp_away = 1 - exp_home

    # Actual outcomes
    if result == 0:
        score_home, score_away = 1, 0
    elif result == 2:
        score_home, score_away = 0, 1
    else:  # Draw
        score_home, score_away = 0.5, 0.5

    # Update ratings
    new_home = elo_home + k * (score_home - exp_home)
    new_away = elo_away + k * (score_away - exp_away)

    return new_home, new_away


# ID → Team mapping
id_map = dict(zip(elo_df['Team ID'], elo_df['Team']))

# Initialize points
points = {
    'Liverpool': 0,
    'Man City': 0,
    'Arsenal ': 0,
    'Man Utd': 0,
    'Chelsea': 0,
    'Tottenham ': 0,
    'Aston Villa': 0,
    'Brighton': 0,
    'Newcastle Utd': 0,
    'Brentford': 0,
    'Fulham': 0,
    'Crystal Palace': 0,
    'Wolves': 0,
    'West Ham': 0,
    'Nottm Forest': 0,
    'Bournemouth ': 0,
    'Everton': 0,
    'Sunderland': 0,
    'Burnley': 0,
    'Leeds': 0
}

# Create a dict to hold current Elo ratings (from df)
elos = {}
for _, row in elo_df.iterrows():
    elos[row['Team']] = row['Elo Rating']

# Extract features and predict
X_new = test[['elo_diff','h2h_index']]
y_new = cat_clf_combo.predict(X_new).flatten()
test['result'] = y_new

# Counters
h_ct, a_ct, d_ct = 0, 0, 0

# Simulate season
for index, row in test.iterrows():
    home_team = id_map.get(row['Home'], row['Home'])
    away_team = id_map.get(row['Away'], row['Away'])
    result = row['result']

    # Use current Elo values
    elo_home = float(elos[home_team])
    elo_away = float(elos[away_team])

    print(f"Match: {home_team} ({elo_home:.1f}) vs {away_team} ({elo_away:.1f}), Result: {'H' if result == 0 else 'A' if result == 2 else 'D'}")

    # Update points
    if result == 0:
        points[home_team] += 3
        h_ct += 1
    elif result == 2:
        points[away_team] += 3
        a_ct += 1
    elif result == 1:
        points[home_team] += 1
        points[away_team] += 1
        d_ct += 1

    # Update Elo ratings
    new_home, new_away = update_elo(elo_home, elo_away, result, k=20)
    elos[home_team] = new_home
    elos[away_team] = new_away

# Final table
sorted_points = dict(sorted(points.items(), key=lambda item: item[1], reverse=True))
print("\n--- Final Points Table ---")
i=1
for team, pts in sorted_points.items():
    print(f"{i}. {team}: {pts} points (Elo {elos[team]:.1f})")
    i+=1

print(f"\nHome Wins: {h_ct}, Away Wins: {a_ct}, Draws: {d_ct}")


Match: Liverpool (1993.0) vs Bournemouth  (1808.0), Result: A
Match: Aston Villa (1873.0) vs Newcastle Utd (1869.0), Result: A
Match: Brighton (1827.0) vs Fulham (1782.0), Result: H
Match: Sunderland (1547.0) vs West Ham (1750.0), Result: A
Match: Tottenham  (1774.0) vs Burnley (1730.0), Result: H
Match: Wolves (1736.0) vs Man City (1960.0), Result: H
Match: Nottm Forest (1803.0) vs Brentford (1811.0), Result: D
Match: Chelsea (1903.0) vs Crystal Palace (1835.0), Result: H
Match: Man Utd (1799.0) vs Arsenal  (1993.0), Result: A
Match: Leeds (1722.0) vs Everton (1794.0), Result: D
Match: West Ham (1754.7) vs Chelsea (1911.1), Result: A
Match: Man City (1944.3) vs Tottenham  (1782.7), Result: H
Match: Bournemouth  (1822.9) vs Wolves (1751.7), Result: H
Match: Brentford (1810.8) vs Aston Villa (1862.9), Result: D
Match: Burnley (1721.3) vs Sunderland (1542.3), Result: H
Match: Arsenal  (1997.9) vs Leeds (1724.0), Result: H
Match: Crystal Palace (1826.9) vs Nottm Forest (1803.2), Result: A

In [116]:
import numpy as np

# Elo update function
def update_elo(elo_home, elo_away, result, k=20):
    """
    Updates Elo ratings after a match.
    result: 'H', 'A', or 'D' (home, away, draw)
    k: K-factor (controls update magnitude)
    """
    # Expected probabilities (logistic model)
    exp_home = 1 / (1 + 10 ** ((elo_away - elo_home) / 400))
    exp_away = 1 - exp_home

    # Actual outcomes
    if result == 0:
        score_home, score_away = 1, 0
    elif result == 2:
        score_home, score_away = 0, 1
    else:  # Draw
        score_home, score_away = 0.5, 0.5

    # Update ratings
    new_home = elo_home + k * (score_home - exp_home)
    new_away = elo_away + k * (score_away - exp_away)

    return new_home, new_away


# ID → Team mapping
id_map = dict(zip(elo_df['Team ID'], elo_df['Team']))

# Initialize points
points = {
    'Liverpool': 0,
    'Man City': 0,
    'Arsenal ': 0,
    'Man Utd': 0,
    'Chelsea': 0,
    'Tottenham ': 0,
    'Aston Villa': 0,
    'Brighton': 0,
    'Newcastle Utd': 0,
    'Brentford': 0,
    'Fulham': 0,
    'Crystal Palace': 0,
    'Wolves': 0,
    'West Ham': 0,
    'Nottm Forest': 0,
    'Bournemouth ': 0,
    'Everton': 0,
    'Sunderland': 0,
    'Burnley': 0,
    'Leeds': 0
}

# Create a dict to hold current Elo ratings (from df)
elos = {}
for _, row in elo_df.iterrows():
    elos[row['Team']] = row['Elo Rating']

# Extract features and predict
X_new = test[['elo_diff','h2h_index']]
y_new = rf.predict(X_new).flatten()
test['result'] = y_new

# Counters
h_ct, a_ct, d_ct = 0, 0, 0

# Simulate season
for index, row in test.iterrows():
    home_team = id_map.get(row['Home'], row['Home'])
    away_team = id_map.get(row['Away'], row['Away'])
    result = row['result']

    # Use current Elo values
    elo_home = float(elos[home_team])
    elo_away = float(elos[away_team])

    print(f"Match: {home_team} ({elo_home:.1f}) vs {away_team} ({elo_away:.1f}), Result: {'H' if result == 0 else 'A' if result == 2 else 'D'}")

    # Update points
    if result == 0:
        points[home_team] += 3
        h_ct += 1
    elif result == 2:
        points[away_team] += 3
        a_ct += 1
    elif result == 1:
        points[home_team] += 1
        points[away_team] += 1
        d_ct += 1

    # Update Elo ratings
    new_home, new_away = update_elo(elo_home, elo_away, result, k=20)
    elos[home_team] = new_home
    elos[away_team] = new_away

# Final table
sorted_points = dict(sorted(points.items(), key=lambda item: item[1], reverse=True))
print("\n--- Final Points Table ---")
i=1
for team, pts in sorted_points.items():
    print(f"{i}. {team}: {pts} points (Elo {elos[team]:.1f})")
    i+=1

print(f"\nHome Wins: {h_ct}, Away Wins: {a_ct}, Draws: {d_ct}")


Match: Liverpool (1993.0) vs Bournemouth  (1808.0), Result: H
Match: Aston Villa (1873.0) vs Newcastle Utd (1869.0), Result: D
Match: Brighton (1827.0) vs Fulham (1782.0), Result: H
Match: Sunderland (1547.0) vs West Ham (1750.0), Result: A
Match: Tottenham  (1774.0) vs Burnley (1730.0), Result: H
Match: Wolves (1736.0) vs Man City (1960.0), Result: A
Match: Nottm Forest (1803.0) vs Brentford (1811.0), Result: D
Match: Chelsea (1903.0) vs Crystal Palace (1835.0), Result: H
Match: Man Utd (1799.0) vs Arsenal  (1993.0), Result: A
Match: Leeds (1722.0) vs Everton (1794.0), Result: D
Match: West Ham (1754.7) vs Chelsea (1911.1), Result: A
Match: Man City (1964.3) vs Tottenham  (1782.7), Result: H
Match: Bournemouth  (1802.9) vs Wolves (1731.7), Result: H
Match: Brentford (1810.8) vs Aston Villa (1872.9), Result: A
Match: Burnley (1721.3) vs Sunderland (1542.3), Result: H
Match: Arsenal  (1997.9) vs Leeds (1724.0), Result: H
Match: Crystal Palace (1826.9) vs Nottm Forest (1803.2), Result: D